In [1]:
import sys
import os
import subprocess
from six import string_types
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy
from skimage import io
from scipy import ndimage
from IPython.display import display
import cv2
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras import regularizers
%matplotlib inline

Using TensorFlow backend.


In [2]:
from keras.models import load_model

In [3]:
PLANET_KAGGLE_ROOT = os.path.abspath("/Users/sunchenxi/Desktop/lab/kaggle/amazon/")
PLANET_KAGGLE_JPEG_DIR = os.path.join(PLANET_KAGGLE_ROOT, 'train-jpg')
PLANET_KAGGLE_LABEL_CSV = os.path.join(PLANET_KAGGLE_ROOT, 'train_v2.csv')
assert os.path.exists(PLANET_KAGGLE_ROOT)
assert os.path.exists(PLANET_KAGGLE_JPEG_DIR)
assert os.path.exists(PLANET_KAGGLE_LABEL_CSV)

# load label13 and weather model

In [5]:
model1 = load_model('my_model.h5')

In [6]:
model2 = load_model('my_model_weather.h5')

# train-jpg labels and images

In [7]:
#labels of train-jpg
labels_df = pd.read_csv(PLANET_KAGGLE_LABEL_CSV)
label_list = []
for tag_str in labels_df.tags.values:
    labels = tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)
for label in label_list:
    labels_df[label] = labels_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)

In [9]:
labels_df.head()

,image_name,tags,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,train_0,haze primary,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,train_1,agriculture clear primary water,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,train_2,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,train_3,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,train_4,agriculture clear habitation primary road,0,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0


In [10]:
# image of train-jpg
image=[0]*1000
n=20000
for t in range(0,1000):
    image[t]=cv2.imread(PLANET_KAGGLE_ROOT+"/train-jpg/train_"+str(n)+".jpg")
    n+=1
resized_img = [np.reshape(cv2.resize(img,(64,64),interpolation=cv2.INTER_AREA),[1,64,64,3]) for img in image]
x=np.vstack((resized_img))

# label13 model predict

In [11]:
# real label
a=labels_df.values[:,3:5]
b=labels_df.values[:,6:11]
c=labels_df.values[:,13:]
label13_real_y=np.hstack((a,b))
label13_real_y=np.hstack((label13_real_y,c))
label13_real_y=label13_real_y[20000:21000]

# predict label
label13_y_pro=model1.predict(x)

In [30]:
# get predict one hot label
label13_y=[] 
for i in range(1000): 
    label13_y.append([]) 
for i in range(0,1000):
    if label13_y_pro[i][0]>0.5:
        label13_y[i].append(1)
    else:
        label13_y[i].append(0)
        
    if label13_y_pro[i][1]>0.1:
         label13_y[i].append(1)
    else:
        label13_y[i].append(0)
        
    if label13_y_pro[i][2]>0.1:
        label13_y[i].append(1)
    else:
        label13_y[i].append(0)
        
    if label13_y_pro[i][3]>0.1:
        label13_y[i].append(1)
    else:
        label13_y[i].append(0)
        
    if label13_y_pro[i][4]>0.1:
        label13_y[i].append(1)
    else:
        label13_y[i].append(0)
        
    if label13_y_pro[i][5]>0.1:
        label13_y[i].append(1)
    else:
        label13_y[i].append(0)
        
    if label13_y_pro[i][6]>0.1:
        label13_y[i].append(1)
    else:
        label13_y[i].append(0)
        
    if label13_y_pro[i][7]>0.1:
        label13_y[i].append(1)
    else:
        label13_y[i].append(0)
        
    if label13_y_pro[i][8]>0.1:
        label13_y[i].append(1)
    else:
        label13_y[i].append(0)
        
    if label13_y_pro[i][9]>0.1:
        label13_y[i].append(1)
    else:
        label13_y[i].append(0)
        
    if label13_y_pro[i][10]>0.1:
        label13_y[i].append(1)
    else:
        label13_y[i].append(0)
        
    if label13_y_pro[i][11]>0.1:
        label13_y[i].append(1)
    else:
        label13_y[i].append(0)
        
    if label13_y_pro[i][12]>0.1:
        label13_y[i].append(1)
    else:
        label13_y[i].append(0)

# weather model predict

In [13]:
#real label
a=labels_df.values[:,5:6]
b=labels_df.values[:,12:13]
c=labels_df.values[:,2:3]
d=labels_df.values[:,11:12]
weather_real_y=np.hstack((a,b))
weather_real_y=np.hstack((weather_real_y,c))
weather_real_y=np.hstack((weather_real_y,d))
weather_real_y=weather_real_y[20000:21000]

# predict label
weather_y_pro=model2.predict(x)

In [21]:
# get predict one hot label
weather_y=[] 
for i in range(1000): 
    weather_y.append([]) 
for i in range(0,1000):
    if np.argmax(weather_y_pro[i],0)==0:
        weather_y[i].append(1)
        weather_y[i].append(0)
        weather_y[i].append(0)
        weather_y[i].append(0)
    elif np.argmax(weather_y_pro[i],0)==1:
        weather_y[i].append(0)
        weather_y[i].append(1)
        weather_y[i].append(0)
        weather_y[i].append(0)
    elif np.argmax(weather_y_pro[i],0)==2:
        weather_y[i].append(0)
        weather_y[i].append(0)
        weather_y[i].append(1)
        weather_y[i].append(0)
    elif np.argmax(weather_y_pro[i],0)==3:
        weather_y[i].append(0)
        weather_y[i].append(0)
        weather_y[i].append(0)
        weather_y[i].append(1)

# combination model predict

### if image have a label cloudy, it cannot have other lables

In [35]:
#predict label
combination_y=np.hstack((label13_y,weather_y))
#if image have a label cloudy, it cannot have other lables
for i in range(len(combination_y)):
    if (combination_y[i][16]==1):
        combination_y[i]=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]  
        
#real label
combination_real_y=np.hstack((label13_real_y,weather_real_y))

In [44]:
print combination_y[0]
print combination_real_y[0]

[1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]


# f2-score

In [127]:
def f2_score():
    f2_score=0.0
    precision=0.0
    recall=0.0
    for i in range(len(combination_y)):
        pre1real1=0.0
        pre1real0=0.0
        pre0real1=0.0
        for j in range(len(combination_y[i])):
            if combination_y[i][j]==combination_real_y[i][j]==1:
                    pre1real1+=1
            if combination_y[i][j]==1 and combination_real_y[i][j]==0:
                    pre1real0+=1
            if combination_y[i][j]==0 and combination_real_y[i][j]==1:
                    pre0real1+=1
        if pre1real1+pre1real0==0 or pre1real1+pre0real1==0:
            f2_score=f2_score+1
        else:
            precision=pre1real1/(pre1real1+pre1real0)
            recall=pre1real1/(pre1real1+pre0real1)
            if(precision+recall==0):
                f2_score+=1
            else:
                f2_score=f2_score+5*precision*recall/(4*precision+recall)
    print (f2_score)/1000

In [128]:
f2_score()

0.866439420212


# test-jpg predict

In [61]:
test_img=[0]*40669
for i in range(0,40669):
    test_img[i]=cv2.imread(PLANET_KAGGLE_ROOT+"/test-jpg/test_"+str(i)+".jpg")
#test_x teat images
test_img_resized = [np.reshape(cv2.resize(img,(64,64),interpolation=cv2.INTER_AREA),[1,64,64,3]) for img in test_img]
test=np.vstack((test_img_resized))

In [62]:
test_label13_y_pro=model1.predict(test)

In [63]:
test_weather_y_pro=model2.predict(test)

In [96]:
test_y=[] 
for i in range(40669): 
    test_y.append([]) 
for i in range(0,40669):
    if test_label13_y_pro[i][0]>0.5:
        test_y[i].append("primary")
    if test_label13_y_pro[i][1]>0.5:
        test_y[i].append("agriculture")
    if test_label13_y_pro[i][2]>0.5:
        test_y[i].append("water")
    if test_label13_y_pro[i][3]>0.5:
        test_y[i].append("habitation")
    if test_label13_y_pro[i][4]>0.5:
        test_y[i].append("road")
    if test_label13_y_pro[i][5]>0.5:
        test_y[i].append("cultivation")
    if test_label13_y_pro[i][6]>0.1:
        test_y[i].append("slash_burn")
    if test_label13_y_pro[i][7]>0.1:
        test_y[i].append("conventional_mine")
    if test_label13_y_pro[i][8]>0.1:
        test_y[i].append("bare_ground")
    if test_label13_y_pro[i][9]>0.1:
        test_y[i].append("artisinal_mine")
    if test_label13_y_pro[i][10]>0.1:
        test_y[i].append("blooming")
    if test_label13_y_pro[i][11]>0.1:
        test_y[i].append("selective_logging")
    if test_label13_y_pro[i][12]>0.1:
        test_y[i].append("blow_down")  
    if np.argmax(test_weather_y_pro[i],0)==0:
        test_y[i].append("clear")
    elif np.argmax(test_weather_y_pro[i],0)==1:
        test_y[i].append("partly_cloudy")
    elif np.argmax(test_weather_y_pro[i],0)==2:
        test_y[i].append("haze")
    elif np.argmax(test_weather_y_pro[i],0)==3:
        test_y[i].append("cloudy")

In [114]:
#image can only have cloudy
for i in range(len(test_y)):
    if (np.array(test_y[i])==["cloudy"]).any():
        test_y[i]=["cloudy"]
for i in range(len(test_y)):
    if (np.array(test_y[i])==["haze"]).all():
        test_y[i]=["cloudy"]

In [115]:
name=[0]*40669
for n in range(0,40669):
    name[n]="test_"+str(n)
final_df = pd.DataFrame({'image_name':name,'tags':test_y})

In [124]:
final_df.head(38)

,image_name,tags
0,test_0,"[primary, clear]"
1,test_1,"[primary, clear]"
2,test_2,"[primary, partly_cloudy]"
3,test_3,"[primary, partly_cloudy]"
4,test_4,"[primary, partly_cloudy]"
5,test_5,"[primary, clear]"
6,test_6,"[primary, agriculture, partly_cloudy]"
7,test_7,"[primary, agriculture, habitation, road, clear]"
8,test_8,"[primary, clear]"
9,test_9,"[primary, haze]"


In [121]:
final_df.to_csv('/Users/sunchenxi/Desktop/lab/code/planet/submission_file.csv', index=False)